# Automating Google Form Filling with Selenium and GPT-4

## Introduction
This notebook demonstrates how to automate the process of filling out Google Forms using Python, Selenium, and OpenAI's GPT-4. We can easily integrate this into our existing Chatbot as well.

The automation script uses Selenium to detect the required fields in a Google Form and then communicates with GPT-4 to generate appropriate input data for these fields. By leveraging GPT-4, we can dynamically construct responses based on the specific form structure, making this approach adaptable and scalable for various forms.

The main objectives of this notebook are:
1. To detect required fields from a Google Form dynamically.
2. To construct form responses using GPT-4 based on the detected fields.
3. To automatically fill the form using Selenium and submit the responses.

**Prerequisites**: Ensure you have Python, Selenium, and OpenAI libraries installed, along with a valid OpenAI API key. Additionally, download the appropriate WebDriver for your browser to interact with the form.

# Install Necessary Libraries

First, install the necessary Python libraries: `selenium` and `openai`.

You can do this by executing the following pip commands:

In [ ]:
# Install Selenium
!pip install selenium
!apt-get update
!apt-get install chromium chromium-driver

# Install OpenAI
!pip install openai


In [3]:
!ls /usr/lib/chromium-browser/

chromedriver


In [14]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

# Import Libraries

We import necessary libraries to interact with the Google Form and to communicate with OpenAI's LLM.

In [36]:
import openai
from time import sleep

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = web_driver()

## API Keys

In [20]:
# Configure your API keys here
OPENAI_API_KEY = ''

# Function to Detect Required Form Fields

We write a function to detect the required fields in a Google Form. The function returns a list of tuples containing the XPath and field type.


In [42]:
def get_required_fields(form_url):
    driver = web_driver()
    required_fields = []

    try:
        driver.get(form_url)
        driver.get(form_url)
        sleep(5)  # To allow page to fully load

        input_elements = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located(
                (By.XPATH, "//input[@aria-required='true'] | //textarea[@aria-required='true']")
            )
        )

        print(input_elements)

        for element in input_elements:
            aria_label = element.get_attribute('aria-label')
            role = element.get_attribute('role')
            xpath = f"//div[@aria-label='{aria_label}']"

            if 'Required' in aria_label:
                required_fields.append((xpath, 'radio' if role == 'radio' else 'text'))

    finally:
        driver.quit()

    return required_fields


# Function to Get Form Data from LLM

This function sends a prompt to the LLM and returns the parsed response, which is a dictionary containing field XPaths and corresponding input data.


In [31]:
def get_form_data_from_llm(prompt):
    """
    Args:
        prompt (str): The prompt to pass to the LLM.
    """
    client = openai.OpenAI(
        api_key=OPENAI_API_KEY
    )

    # Construct the conversation
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    # Create a completion
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    # Extract the last response from the model
    generated_text = response['choices'][0]['message']['content'].strip()
    return eval(generated_text)  # Caution: Be aware of security risks with eval


# Function to Fill Google Form

This function receives the form data and uses Selenium to fill out the Google Form automatically.

In [38]:
def fill_google_form(form_url, form_data):
    driver = web_driver()

    try:
        driver.get(form_url)
        sleep(5)  # To allow page to fully load

        wait = WebDriverWait(driver, 20)

        for field_xpath, value in form_data.items():
            field_element = wait.until(EC.presence_of_element_located((By.XPATH, field_xpath)))
            if field_element.get_attribute('role') == 'radio':
                field_element.click()
            else:
                field_element.send_keys(value)

        submit_button_xpath = '//span[contains(text(), "Submit")]'
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_button_xpath)))
        submit_button.click()

    finally:
        driver.quit()


## Function to Automate Google Form Filling

Automates the process of detecting required fields in a Google Form, generating data with an LLM, and filling the form.

In [33]:
def automate_google_form_filling(form_url):
    """
    Args:
        form_url (str): The URL of the Google Form.
        webdriver_path (str): Path to the WebDriver executable.
    """
    # Detect required fields
    required_fields = get_required_fields(form_url)

    # Construct the prompt for the LLM
    prompt = "Generate a dictionary in Python format where keys are XPaths and values are input data:\n"
    for xpath, field_type in required_fields:
        prompt += f"{xpath}: "

    # Generate form data using the LLM
    form_data = get_form_data_from_llm(prompt)

    # Fill the Google Form with the generated data
    fill_google_form(form_url, form_data)


# Main Function to Execute the Process

In this section, we execute the entire process of detecting required fields, constructing data with LLM, and filling the Google Form.

In [44]:
form_url = ''
automate_google_form_filling(form_url)